In [ ]:
# Modules
import warnings
warnings.simplefilter('ignore') # filter some warning messages

import numpy as np
import pandas as pd
import xarray as xr
from calendar import monthrange

In [ ]:
# anomalies of SST
dsa = xr.open_mfdataset('./OISSTa/OISST_*.nc')
iy=dsa['time.year'].min().data
fy=dsa['time.year'].max().data
dsa

In [ ]:
# SST climatology
dsc = xr.open_dataset('./OISSTa/OISSTdV2_KateHigh_Cal.nc')
dsc.close()
dscm = dsc.resample(time='1M').mean(keep_attrs=True,skipna=False)
dscm

In [ ]:
# read whale data in csv format
mam = pd.read_csv('stranding data for marisol.csv')
mam['SSTa (C)']=np.nan
mam['SST (C)']=np.nan
#list(mam)

In [ ]:
# collocate data
# list the nmber of records processed
for row in mam.iterrows():
    lats = row[1]['georeferenced latitude']
    lons = row[1]['georeferenced longitude']
    years = row[1]['year']
    months = row[1]['month']
        
    # colocate data
    fecha1 = str(years)+'-'+str(months).zfill(2)+'-01'
    edy = monthrange(years, months)[1]
    fecha2 = str(years)+'-'+str(months).zfill(2)+'-'+str(edy).zfill(2)
    #print(fecha1, fecha2)
    t = dsa.sel(lat=float(lats),lon=float(lons)+360, method='ffill', tolerance=0.3)
    t2 = t.sel(time=slice(fecha1,fecha2)).anom.mean()
    print('.')
    mam.at[row[0],'SSTa (C)'] = t2.values
    
    # values
    t3 = dscm.sel(lat=float(lats),lon=float(lons)+360, method='ffill', tolerance=0.3)
    t4 = t3.sst[months-1].values
    mam.at[row[0],'SST (C)'] = t2.values + t4

In [ ]:
# save the file
mam[{'georeferenced longitude','georeferenced latitude','month','year','SSTa (C)'}]
mam.to_csv('stranding data collocated SST 29March2022.csv')